In [13]:
"""
Duplicate test monitor: cholesterol (>= 180d) and LFT (>= 90d)
Flags unnecessary repeats unless ordering_reason is in allowed_exceptions
"""

import pandas as pd 
import datetime as datetime 

#load data - ensure date column read as date
patients = pd.read_csv(r"C:\Users\kez_g\Desktop\Coding & Data Analysis\duplicate_test_monitor\patients.csv", parse_dates = ['dob'])
tests = pd.read_csv(r"C:\Users\kez_g\Desktop\Coding & Data Analysis\duplicate_test_monitor\tests.csv", parse_dates = ['test_date'])

#Define rules - store in dictionary
interval_rules = {
    'cholesterol': 180,
    'lft': 90
}

allowed_exceptions = {'abnormal', 'urgent', 'cancer pathway', 'acute presentation'}                        #set - unordered/ unique - fast processing for 'in' checks

#Sort and merge 
tests.sort_values(['patient_id', 'test_name', 'test_date'], inplace = True)

#Check for duplicates
flags = []

for pid, group in tests.groupby(['patient_id', 'test_name']):
    group = group.sort_values('test_date')                     #df with all the rows that match the groupby criteria, ordered from oldest to newest 
    for i in range(1, len(group)):                              #iterate through the rows starting at the second row coz we are comparing
        current = group.iloc[i]
        previous = group.iloc[i - 1]                               #create 2x series of the current and previous rows 
        days_since = (current.test_date - previous.test_date).days                 #subtracting two dates creates a Timedelta object which has the .days attribute 
        test_type = current.test_name.strip().lower()                             #syntax for series - short for current['test_type']

    if test_type in interval_rules:
        min_days = interval_rules[test_type]
        if days_since < min_days:
            reason = str(current.ordering_reason).strip().lower()
            if reason not in allowed_exceptions:
                flags.append({
                    'patient_id': current.patient_id,
                    'test_name': test_type,
                    'prev_date': previous.test_date.date(),
                    'curr_date': current.test_date.date(),
                    'days_between': days_since,
                    'prev_reason': previous.ordering_reason,
                    'curr_reason': current.ordering_reason
                })
                
duplicates = pd.DataFrame(flags)
print(duplicates)



   patient_id    test_name   prev_date   curr_date  days_between  \
0        P010          lft  2024-04-08  2024-05-07            29   
1        P011  cholesterol  2024-01-30  2024-06-12           134   
2        P011  cholesterol  2024-01-30  2024-06-12           134   
3        P012  cholesterol  2024-04-21  2024-04-22             1   
4        P012  cholesterol  2024-04-21  2024-04-22             1   
5        P013          lft  2024-04-24  2024-06-16            53   
6        P013          lft  2024-04-24  2024-06-16            53   
7        P013          lft  2024-04-24  2024-06-16            53   
8        P018  cholesterol  2024-06-01  2024-06-11            10   
9        P018  cholesterol  2024-06-01  2024-06-11            10   
10       P019  cholesterol  2024-01-30  2024-03-13            43   
11       P020          lft  2024-05-04  2024-06-23            50   
12       P020          lft  2024-05-04  2024-06-23            50   
13       P028          lft  2024-05-05  2024-06-